# **1.Time Benchmark**

In [ ]:
import pandas as pd

df = pd.read_csv("../../Data/AAM/unbalance/mapping_times.csv", index_col=0)
df.rename(columns={"Unnamed: 0": "dataset"}, inplace=True)
df["Number of Reactions"] = [273, 382, 3000, 1758, 491]
df

In [ ]:
import copy

df.rename(
    columns={
        "rxn_mapper": r"$\texttt{RXNMapper}$",
        "graphormer": r"$\texttt{GraphormerMapper}$",
        "local_mapper": r"$\texttt{LocalMapper}$",
        "rdt": r"$\texttt{RDT}$",
    },
    inplace=True,
)

df_time = copy.deepcopy(df)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from _analysis._aam_analysis import plot_reaction_times

plt.rc("text", usetex=True)
plt.rc("text.latex", preamble=r"\usepackage{amsmath}")
sns.set_theme(style="darkgrid")
# Example of how to use the modified function in a subplot


# Example of how to use the modified function in a subplot
fig, ax = plt.subplots(figsize=(12, 6))

plot_reaction_times(df_time, ax)

# plt.tight_layout()
fig.savefig("./fig/aam_time_benchmark.pdf", dpi=600, bbox_inches="tight", pad_inches=0)
plt.show()

# **2. Success rate Benchmark**

In [ ]:
import re


def calculate_mapping_failures(data):
    """
    Calculates the number of failed mappings based on the absence of atom maps in the reaction data.
    Args:
    data (list of dicts): A list of dictionaries where each dictionary represents reaction data.

    Returns:
    dict: A dictionary containing the number of fails and success rates for each mapper type.
    """

    # Keys to evaluate
    keys_to_check = ["rxn_mapper", "graphormer", "local_mapper", "rdt"]

    # Initialize results dictionary to store fails and successes
    results = {key: {"fails": 0, "successes": 0} for key in keys_to_check}

    # Regex pattern to find atom maps
    atom_map_pattern = re.compile(r":\d+")

    # Process each dictionary in the list
    for entry in data:
        for key in keys_to_check:
            # Get the reaction data
            reaction_data = entry.get(key, "")

            # Check if the reaction contains any atom maps
            if re.search(atom_map_pattern, reaction_data):
                results[key]["successes"] += 1
            else:
                results[key]["fails"] += 1

    # Prepare aggregate results to provide total fails and success rates
    aggregate_results = {}
    for key in keys_to_check:
        total_fails = results[key]["fails"]
        total_successes = results[key]["successes"]
        total_attempts = total_fails + total_successes
        success_rate = (total_successes / total_attempts) if total_attempts > 0 else 0

        aggregate_results[f"{key}_number_fails"] = int(total_fails)
        aggregate_results[f"{key}_success_rate"] = round(
            success_rate * 100, 2
        )  # Express as percentage

    return aggregate_results

In [ ]:
import sys

sys.path.append("../../")
from syntemp.SynUtils.utils import load_database

data = load_database("../../Data/AAM/unbalance/golden/golden_aam_reactions.json.gz")

In [ ]:
from syntemp.SynAAM.aam_validator import AAMValidator

results, _ = AAMValidator.validate_smiles(
    data=data,
    ground_truth_col="ground_truth",
    mapped_cols=["rxn_mapper", "graphormer", "local_mapper", "rdt"],
    check_method="RC",
    ignore_aromaticity=False,
    n_jobs=4,
    verbose=2,
    ensemble=True,
    strategies=[
        ["rxn_mapper", "graphormer", "local_mapper"],
        ["rxn_mapper", "graphormer", "local_mapper", "rdt"],
    ],
    ignore_tautomers=False,
)

In [ ]:
import sys

sys.path.append("../..")
from syntemp.SynUtils.utils import load_database

data_paths = {
    "ecoli": "../../Data/AAM/unbalance/ecoli/ecoli_aam_reactions.json.gz",
    "recon3d": "../../Data/AAM/unbalance/recon3d/recon3d_aam_reactions.json.gz",
    "uspto_3k": "../../Data/AAM/unbalance/uspto_3k/uspto_3k_aam_reactions.json.gz",
    "golden": "../../Data/AAM/unbalance/golden/golden_aam_reactions.json.gz",
    "natcomm": "../../Data/AAM/unbalance/natcomm/natcomm_aam_reactions.json.gz",
}

# Dictionary to hold the results for each dataset
results_dict = {}

# Process each dataset
for dataset_name, filepath in data_paths.items():
    # Load the data
    data = load_database(filepath)

    # Calculate fails and success rates
    results = calculate_mapping_failures(data)

    # Store the results
    results_dict[dataset_name] = results

# Convert the results dictionary to a DataFrame
df_results = pd.DataFrame.from_dict(results_dict, orient="index")

# Display or save the DataFrame
df_results.T

# **3. Accuracy Benchmark**

In [ ]:
import sys
import pandas as pd

sys.path.append("../..")
from syntemp.SynAAM.aam_validator import AAMValidator

In [ ]:
data_paths = {
    "ecoli": "../../Data/AAM/unbalance/ecoli/ecoli_aam_reactions.json.gz",
    "recon3d": "../../Data/AAM/unbalance/recon3d/recon3d_aam_reactions.json.gz",
    "uspto_3k": "../../Data/AAM/unbalance/uspto_3k/uspto_3k_aam_reactions.json.gz",
    "golden": "../../Data/AAM/unbalance/golden/golden_aam_reactions.json.gz",
    "natcomm": "../../Data/AAM/unbalance/natcomm/natcomm_aam_reactions.json.gz",
}
results_dict = {}

# Process each dataset
for dataset_name, filepath in data_paths.items():
    # Load the data
    data = load_database(filepath)

    # Calculate fails and success rates
    results, _ = AAMValidator.validate_smiles(
        data=data,
        ground_truth_col="ground_truth",
        mapped_cols=["rxn_mapper", "graphormer", "local_mapper", "rdt"],
        check_method="RC",
        ignore_aromaticity=False,
        n_jobs=4,
        verbose=2,
        ensemble=True,
        strategies=[
            ["rxn_mapper", "graphormer", "local_mapper"],
            ["rxn_mapper", "graphormer", "local_mapper", "rdt"],
        ],
        ignore_tautomers=False,
    )

    # Store the results
    results_dict[dataset_name] = results

bio = []
for dataset_name, filepath in data_paths.items():
    # Load the dataset
    single = load_database(filepath)

    # Extend the bio list if the dataset is either 'ecoli' or 'recon3d'
    if dataset_name in ["ecoli", "recon3d"]:
        bio.extend(single)
results, _ = AAMValidator.validate_smiles(
    data=bio,
    ground_truth_col="ground_truth",
    mapped_cols=["rxn_mapper", "graphormer", "local_mapper", "rdt"],
    check_method="RC",
    ignore_aromaticity=False,
    n_jobs=4,
    verbose=2,
    ensemble=True,
    strategies=[
        ["rxn_mapper", "graphormer", "local_mapper"],
        ["rxn_mapper", "graphormer", "local_mapper", "rdt"],
    ],
    ignore_tautomers=False,
)
results_dict["Biochemical"] = results


chem = []
for dataset_name, filepath in data_paths.items():
    # Load the dataset
    single = load_database(filepath)

    # Extend the bio list if the dataset is either 'ecoli' or 'recon3d'
    if dataset_name in ["golden", "natcomm", "uspto_3k"]:
        chem.extend(single)

results, _ = AAMValidator.validate_smiles(
    data=chem,
    ground_truth_col="ground_truth",
    mapped_cols=["rxn_mapper", "graphormer", "local_mapper", "rdt"],
    check_method="RC",
    ignore_aromaticity=False,
    n_jobs=4,
    verbose=2,
    ensemble=True,
    strategies=[
        ["rxn_mapper", "graphormer", "local_mapper"],
        ["rxn_mapper", "graphormer", "local_mapper", "rdt"],
    ],
    ignore_tautomers=False,
)
results_dict["Chemical"] = results

In [ ]:
# Initialize an empty DataFrame to hold all data
import pandas as pd

final_df = pd.DataFrame()

# Process each data type
for data_type, records in results_dict.items():
    # Create a DataFrame
    df = pd.DataFrame(records)

    # Add columns for accuracy and success_rate specific to the data type
    df["accuracy_col"] = data_type + "_accuracy"
    df["success_col"] = data_type + "_success_rate"
    df["Accuracy"] = round(df["accuracy"] * 100, 2)
    df["Success Rate"] = round(df["success_rate"], 2)

    # Pivot the DataFrame
    df_pivot = df.pivot(index="mapper", columns="accuracy_col", values="Accuracy").join(
        df.pivot(index="mapper", columns="success_col", values="Success Rate")
    )

    # Merge with the final DataFrame
    if final_df.empty:
        final_df = df_pivot
    else:
        final_df = final_df.join(df_pivot)

# Reset index to make 'mapper' a column
final_df.reset_index(inplace=True)
final_df = final_df.reindex([5, 2, 3, 4, 0, 1])
final_df = final_df.reset_index(drop=True)

In [ ]:
aam_json = final_df.to_dict(orient="records")

In [ ]:
from syntemp.SynUtils.utils import save_database

save_database(aam_json, "../../Data/AAM/unbalance/aam_benchmark.json.gz")

## 3.0 Load results

In [ ]:
import sys
import pandas as pd

sys.path.append("../../")
from syntemp.SynUtils.utils import load_database

final_df = pd.DataFrame(load_database("../../Data/AAM/unbalance/aam_benchmark.json.gz"))

In [ ]:
final_df[
    [
        "mapper",
        "Biochemical_accuracy",
        "Biochemical_success_rate",
        "Chemical_accuracy",
        "Chemical_success_rate",
    ]
]

## 3.1 Heatmap

In [ ]:
data_visual = final_df[
    [
        "mapper",
        "ecoli_accuracy",
        "recon3d_accuracy",
        "uspto_3k_accuracy",
        "golden_accuracy",
        "natcomm_accuracy",
        "Biochemical_accuracy",
        "Chemical_accuracy",
    ]
]
data_visual.rename(
    {
        "ecoli_accuracy": "ecoli",
        "recon3d_accuracy": "recon3d",
        "uspto_3k_accuracy": "uspto_3k",
        "golden_accuracy": "golden",
        "natcomm_accuracy": "natcomm",
        "Biochemical_accuracy": "Biochemical",
        "Chemical_accuracy": "Chemical",
    },
    axis=1,
    inplace=True,
)

In [ ]:
data_visual["mapper"] = [
    r"$\texttt{RXNMapper}$",
    r"$\texttt{GraphormerMapper}$",
    r"$\texttt{LocalMapper}$",
    r"$\texttt{RDT}$",
    r"$\textit{Ensemble_1}$",
    r"$\textit{Ensemble_2}$",
]

In [ ]:
data_visual

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.rc("text", usetex=True)
plt.rc("text.latex", preamble=r"\usepackage{amsmath}")
sns.set_theme(style="darkgrid")
from _analysis._aam_analysis import plot_heatmap

fig = plot_heatmap(data_visual, "./fig/aam_accuracy_heatmap.pdf")
plt.show()

## 3.2 Barplot

In [ ]:
df = final_df.copy()

In [ ]:
from _analysis._aam_analysis import plot_accuracy_success_rate_subplot
import seaborn as sns
import matplotlib.pyplot as plt

plt.rc("text", usetex=True)
plt.rc("text.latex", preamble=r"\usepackage{amsmath}")
sns.set_theme(style="darkgrid")

fig, axes = plt.subplots(1, 2, figsize=(18, 8))
plot_accuracy_success_rate_subplot(
    df,
    ["Chemical_accuracy", "Biochemical_accuracy"],
    ["Chemical_success_rate", "Biochemical_success_rate"],
    [r"$\textit{Chemical dataset}$", r"$\textit{Biochemical dataset}$"],
    axes,
)

In [ ]:
from _analysis._aam_analysis import (
    plot_accuracy_success_rate_subplot,
    plot_reaction_times,
)
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec


accuracy_cols = ["Chemical_accuracy", "Biochemical_accuracy"]
success_cols = ["Chemical_success_rate", "Biochemical_success_rate"]
titles = [r"$\textit{B. Chemical dataset}$", r"$\textit{C. Biochemical dataset}$"]

df = final_df.copy()
df["mapper"] = [
    r"$\texttt{RXNMapper}$",
    r"$\texttt{GraphormerMapper}$",
    r"$\texttt{LocalMapper}$",
    r"$\texttt{RDT}$",
    r"$\textit{Ensemble_1}$",
    r"$\textit{Ensemble_2}$",
]

# Create a 2x2 subplot layout
fig = plt.figure(figsize=(18, 16))
gs = gridspec.GridSpec(2, 2, height_ratios=[1, 1], width_ratios=[1, 1])

# First row: reaction times plot (spans the entire width)
ax0 = plt.subplot(gs[0, :])
plot_reaction_times(df_time, ax0)

# Second row: accuracy and success rate subplots
ax1 = plt.subplot(gs[1, 0])
ax2 = plt.subplot(gs[1, 1])
plot_accuracy_success_rate_subplot(df, accuracy_cols, success_cols, titles, [ax1, ax2])

# Adjust layout to prevent overlap
plt.tight_layout()

# Display the combined plot
fig.savefig(
    "./fig/aam_time_data_benchmark.pdf", dpi=600, bbox_inches="tight", pad_inches=0
)
plt.show()